In [1]:
import cv2
import numpy as np
import imageio
import matplotlib.pyplot as plt



Features to add:
- Change appearance mode button (light/dark)
- Video frame slider (across the bottom of the frame)
- Options panel (right side) **WIP**
- Video filepath (read)
- Exporting (+ filepath)
- Frame skip rate
- User-adjusted image size
- Refresh image after the parameters have been adjusted (without skipping to next frame)

In [8]:
import tkinter
import tkinter.messagebox
import customtkinter
import cv2
from PIL import Image, ImageTk
import time
from screeninfo import get_monitors
import numpy as np

from src.imageProcessing import CannyLines, MixedCannyLines, blurImages


customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"


class GUI(customtkinter.CTk):
    def __init__(self):

        self.filePath = 'Input Videos\Arsenal_goal.mp4'
        self.cap = cv2.VideoCapture(self.filePath)

        # get first frame
        ret, frame = self.cap.read()
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (720,480))
        self.currentImage = frame
        self.prevImage =  np.zeros_like(frame, dtype=np.uint8)


        super().__init__()

        # Images settings
        self.imgParams = {'length' : int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT)),
                        'current frame' : 0,
                        'playing' : False, 
                        'blurring' : False, 
                            'blur coeff' : 0.2,
                        'multiLine' : False,
                            'lower bound' : 0, 
                            'upper bound' : 60, 
                            'dilation' : 1,
                            'ranges' : [0,100,200,255], 
                            'dilations' : [1,2,4],
                        'fadeout' : False}

        # configure window

        self.title("Graphic design is my passion.py")
        monitors = get_monitors()
        for m in monitors:
            if m.is_primary == True:
                width = m.width, 
                height = m.height

        self.geometry(f"{1100}x{580}")

        # configure grid layout (4x4)
        #self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((0, 2), weight=0)
        self.grid_columnconfigure((1), weight=1)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        # SIDEBAR (column 0, all rows)
        self.sidebar_frame = customtkinter.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        text_var = tkinter.StringVar(value="Play Clip")
        self.play_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.play_switch_func())
        self.play_switch.grid(row=0, column=0, pady=30, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="MultiLine")
        self.multiLine_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.multiLine_switch.grid(row=1, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="Blurring")
        self.blurring_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.blurring_switch.grid(row=2, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="Fadeout")
        self.fadeout_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.fadeout_switch.grid(row=3, column=0, pady=10, padx=20, sticky="n")

        # IMAGE FRAME
        self.image_frame = customtkinter.CTkFrame(self)
        self.image_frame.grid(row=0, column=1, rowspan=4, columnspan=2, padx=(10, 10), pady=(40, 40), sticky="nsew")
    
        imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame))    
        self.panel = tkinter.Label(master=self.image_frame, image=imgtk, width=720, height=480)
        self.panel.image = imgtk
        self.panel.grid(row=1,column=1,rowspan=3,columnspan=1)


        # RIGHT-SIDE PANEL (tabs)
        self.tabview = customtkinter.CTkTabview(self, width=300)
        self.tabview.grid(row=0, column=2, rowspan=4, padx=(10, 10), pady=(0, 10), sticky="nsew")
        
        self.tabview.add("Edges")
        self.tabview.add("Blurring")
        self.tabview.add("Fadeout")
        # EDGES TAB
        self.EdgeVis = None
        self.edges_tab_appearance()
        
        
        # BLURRING TAB
        self.labelblur = customtkinter.CTkLabel(master=self.tabview.tab("Blurring"), text="% of Previous Frame to Overlay:")
        self.labelblur.grid(row=0, column=0)

        self.blur_coeff_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Blurring"), textvariable=tkinter.StringVar(value=self.imgParams['blur coeff']), width=40)
        self.blur_coeff_txtbx.grid(row=1, column=0)
        self.blur_coeff_txtbx.bind("<KeyRelease>", self.blur_params)

        self.blur_slider = customtkinter.CTkSlider(master=self.tabview.tab("Blurring"), from_=0, to=1, number_of_steps=10)
        self.blur_slider.grid(row=3, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
        #self.blur_slider.configure(command=lambda: self.blur_params())
        self.blur_slider.bind("<ButtonRelease-1>", self.blur_params)



    def blur_params(self, value):
        slider_val = round(float(self.blur_slider.get()), 1)
        txtbx_val = round(float(self.blur_coeff_txtbx.get()), 1)
        #round(number,1)

        if slider_val != self.imgParams['blur coeff']:
            self.imgParams['blur coeff'] = slider_val
            self.blur_coeff_txtbx.configure(textvariable=tkinter.StringVar(value=slider_val))
        elif txtbx_val != self.imgParams['blur coeff']:
            self.imgParams['blur coeff'] = txtbx_val
            self.blur_slider.set(textvariable=tkinter.StringVar(value=txtbx_val))




    def update_params(self):
        #print(self.imgParams)
        if self.imgParams['multiLine'] == True:
            self.imgParams['ranges'] = [int(self.edge_left_bound.get()), int(self.edge_middle_left_bound.get()), int(self.edge_middle_right_bound.get()), int(self.edge_right_bound.get())]
            self.imgParams['dilations'] = [int(self.edge_left_line.get()), int(self.edge_middle_line.get()), int(self.edge_right_line.get())]
        else:
            self.imgParams['lower bound'] = int(self.lower_bound_txtbx.get())
            self.imgParams['upper bound'] = int(self.upper_bound_txtbx.get())
            self.imgParams['dilation'] = int(self.line_dil_txtbx.get())
        
    

    def edges_tab_appearance(self):
        # change Edge tab to show options for multipline line thicknesses
        if self.imgParams['multiLine'] == True and self.EdgeVis != "multi": 
            if self.EdgeVis == 'single':
                self.edge_LB_label.grid_remove()
                self.lower_bound_txtbx.grid_remove()
                self.edge_UB_label.grid_remove()
                self.upper_bound_txtbx.grid_remove()
                self.edge_dil_label.grid_remove()
                self.line_dil_txtbx.grid_remove()
                self.update_button_edges.grid_remove()

            self.tabview.tab("Edges").grid_columnconfigure((0,1,2,3), weight=1)
            self.tabview.tab("Edges").grid_rowconfigure((0,1,2,3,4,5,6), weight=0)


            # Add stuff here!
            self.edge_bounds_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Bound Ranges:")
            self.edge_bounds_label.grid(row=1, column=1, columnspan=6)
            self.edge_left_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][0]))
            self.edge_left_bound.grid(row=2,column=0,pady=10)
            self.edge_middle_left_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][1]))
            self.edge_middle_left_bound.grid(row=2,column=2,pady=10)
            self.edge_middle_right_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][2]))
            self.edge_middle_right_bound.grid(row=2,column=4,pady=10)
            self.edge_right_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][3]))
            self.edge_right_bound.grid(row=2,column=6,pady=10)

            self.edge_lines_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Line Thicknesses:")
            self.edge_lines_label.grid(row=3, column=1, columnspan=6)
            self.edge_left_line = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['dilations'][0]))
            self.edge_left_line.grid(row=4,column=1,pady=10)
            self.edge_middle_line = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['dilations'][1]))
            self.edge_middle_line.grid(row=4,column=3,pady=10)
            self.edge_right_line = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['dilations'][2]))
            self.edge_right_line.grid(row=4,column=5,pady=10)

            self.update_button_edges = customtkinter.CTkButton(master=self.tabview.tab("Edges"), text="Update values", fg_color="transparent", 
                                                                border_width=2, text_color=("gray10", "#DCE4EE"), command=lambda: self.update_params())
            self.update_button_edges.grid(row=6, column=1, columnspan=6, pady=30)

            self.EdgeVis = "multi"

        # change Edge tab to show options for single line thickness
        elif self.imgParams['multiLine'] == False and self.EdgeVis != "single":
            if self.EdgeVis == 'multi':
                self.edge_bounds_label.grid_remove()
                self.edge_left_bound.grid_remove()
                self.edge_middle_left_bound.grid_remove()
                self.edge_middle_right_bound.grid_remove()
                self.edge_right_bound.grid_remove()

                self.edge_lines_label.grid_remove()
                self.edge_left_line.grid_remove()
                self.edge_middle_line.grid_remove()
                self.edge_right_line.grid_remove()

                self.update_button_edges.grid_remove()
            
            self.tabview.tab("Edges").grid_columnconfigure((0), weight=1)
            self.tabview.tab("Edges").grid_columnconfigure((1,2,3), weight=0)
            self.tabview.tab("Edges").grid_rowconfigure((0,1,2,3,4), weight=0)
            
            self.edge_LB_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Lower Bound:")
            self.edge_LB_label.grid(row=0, column=0)
            self.lower_bound_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=self.imgParams['lower bound']))
            self.lower_bound_txtbx.grid(row=1, column=0, pady=0, padx=10)
            self.edge_UB_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Upper Bound:")
            self.edge_UB_label.grid(row=2, column=0)
            self.upper_bound_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=self.imgParams['upper bound']))
            self.upper_bound_txtbx.grid(row=3, column=0, pady=0, padx=10)
            self.edge_dil_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Dilation:")
            self.edge_dil_label.grid(row=4, column=0)
            self.line_dil_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=self.imgParams['dilation']))
            self.line_dil_txtbx.grid(row=5, column=0, pady=0, padx=10)
            self.update_button_edges = customtkinter.CTkButton(master=self.tabview.tab("Edges"), text="Update values", fg_color="transparent", 
                                                                border_width=2, text_color=("gray10", "#DCE4EE"), command=lambda: self.update_params())
            self.update_button_edges.grid(row=6, column=0, pady=30)

            self.EdgeVis = "single"



    # formulate a better while loop than this ? 
    def left_panel_switches(self):
        self.imgParams['multiLine'] = self.multiLine_switch.get()
        self.imgParams['blurring'] = self.blurring_switch.get()
        self.imgParams['fadeout'] = self.fadeout_switch.get()
        self.edges_tab_appearance()
        #print(self.imgParams['fadeout'])

    
    def play_switch_func(self):
        # get their values
        update = self.play_switch.get()
        if update: self.refresh_frame()
        

    def refresh_frame(self):
        # load the next image here
        #print("refreshing")
        ret, frame = self.cap.read()
        if ret:
            #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(frame, (720,480))
            self.augmentImage(frame)  
            self.update() 
            #time.sleep(0.05)       
            if self.play_switch.get(): self.refresh_frame() # call itself again, if we want that
        else:
            self.cap.release() # finished clip (restart it)
            self.cap = cv2.VideoCapture(self.filePath)
            if self.play_switch.get(): self.refresh_frame()


    def setFrame(self, frame): 
        imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame))    
        self.panel.configure(image=imgtk)
        self.panel.image = imgtk
        #print("set new")

    def augmentImage(self, frame):
        if self.imgParams['multiLine'] == True:
            canny_lines = MixedCannyLines(frame, ranges=self.imgParams['ranges'], dilations=self.imgParams['dilations'])
        else:
            canny_lines = CannyLines(frame, lower_bound=self.imgParams['lower bound'],upper_bound=self.imgParams['upper bound'], dilation=self.imgParams['dilation'])

        if self.imgParams['blurring'] == True:
            result = blurImages(canny_lines, self.prevImage, blur=self.imgParams['blur coeff'])
        else:
            result = canny_lines

        self.setFrame(result)
        self.prevImage = result

    

app = GUI()
app.mainloop()

param= 0.7
param= 0.2
param= 0.9
param= 0.0
param= 1.0
param= 0.0
param= 1.0
param= 0.0
param= 0.9


Exception in Tkinter callback
Exception ignored in: <function PhotoImage.__del__ at 0x0000022880349240>
Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\site-packages\PIL\ImageTk.py", line 133, in __del__
    name = self.__photo.name
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'
Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\site-packages\customtkinter\windows\widgets\ctk_switch.py", line 405, in toggle
    self._command()
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_24932\3559428481.py", line 70, in <lambda>
    self.play_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.play_switch_func())
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_24932\3559428481.py", line 249, in

In [9]:
from tkinter import *
  
# Create a GUI app
app=Tk()
  
# Set the title and geometry of the window
app.title('Remove Tkinter Widget')
app.geometry("600x400")
  
# Make a function to remove the widget
def remove_widget():
   label.place_forget()
  
# Create a label widget to display text
label=Label(app, text="Tkinter Widget", font='Helvetica 20 bold')
label.place(relx=0.5, rely=0.3, anchor=CENTER)
  
# Create a button to remove text
button=Button(app, text="Remove Widget", command=remove_widget)
button.place(relx=0.5, rely=0.7, anchor=CENTER)
  
# Make infinite loop for displaying app 
# on the screen
app.mainloop()

In [16]:
from tkinter import *
import threading
import time

class gui:
    def __init__(self, window):
        # play button
        self.play_frame = Frame(master=window, relief=FLAT, borderwidth=1)
        self.play_frame.grid(row=0, column=0, padx=1, pady=1)
        self.play_button = Button(self.play_frame, text="play", fg="blue", command=lambda: self.play(1))
        self.play_button.pack()
        # stop button
        self.stop_frame = Frame(master=window, relief=FLAT, borderwidth=1)
        self.stop_frame.grid(row=0, column=2, padx=1, pady=1)
        self.stop_button = Button(self.stop_frame, text="stop", fg="red", command=lambda: self.play(0))
        self.stop_button.pack()

    def play(self, switch):
        a.set(switch) # update 'a'
        print(a.get())

root = Tk()

a = IntVar(value=0)

def tester(trig):
    while True:
        value = trig.get()
        if value == 1:
            time.sleep(0.5)
            print ("running")
        elif value == 0:
            time.sleep(0.5)
            print ("not running")

t1 = threading.Thread (target = tester, args = [a], daemon = True)
t1.start()

app = gui(root)
root.mainloop()

not running
not running
not running
not running
not running
not running
not running
not running
not running
not running
0
not running
not running
not running
not running
not running
1
not running
running
running
running
running
running
running
running
running
running
running
running
running


running


Exception in thread Thread-3 (tester):
Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_8148\84278952.py", line 28, in tester
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\tkinter\__init__.py", line 568, in get
    value = self._tk.globalgetvar(self._name)
RuntimeError: main thread is not in main loop
